In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from random import sample
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
import os
import random
from surprise.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
import gzip
import json

In [3]:
raw_data = pd.read_csv("Data/AMAZON_FASHION.csv", names=['Item', 'User', 'Rating', 'Timestamp'])
data = raw_data.sample(n=30000)
data.drop(['Timestamp'], axis=1,inplace=True)
data = data[['User', 'Item', 'Rating']]

In [4]:
data

,User,Item,Rating
733879,ACJUNN3WM5V5W,B017ALT8KE,5.0
405473,A2J1E3D25AYXZ2,B001D301H0,5.0
785550,A1LH91HW8LEMHP,B01B4X18LG,5.0
513651,A3NHSJQ5QHYIWA,B00JV0KGGI,4.0
33837,A3PPAAKO215G16,B0014HA6VG,4.0
...,...,...,...
679691,A1JKR74UXCLVQN,B012JNQH2U,5.0
118884,A3IM1IFCU6QIEA,B00CTXVBPY,3.0
689903,A2F880Z6TN9N6H,B013WOCD7S,4.0
736318,AC4EFAU3FTMWA,B017EQYASU,5.0


In [5]:
new_df=data.head(10000)
ratings_matrix = new_df.pivot_table(values='Rating', index='User', columns='Item', fill_value=0)
ratings_matrix.head()

Item,B000078QKW,B00007GDFV,B000085HZL,B00008JOQI,B00008JPRZ,B00008JQF0,B00008JRUJ,B00008JVTT,B0000AWX3V,B0000CH6PW,...,B01HHBRNYK,B01HI7K476,B01HIVIUAU,B01HIWA9CQ,B01HIZYZ1O,B01HJ0KHEM,B01HJ0SY4C,B01HJ3O7AY,B01HJ5WLQE,B01HJEOBUO
User,,,,,,,,,,,,,,,,,,,,,
A00248626G99Y8GVGKKQ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0254933A6GXCEJ2WU0K,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A1006WV7OJQAWN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A101EHUSV30KDM,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A101P4QD1IF90E,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X = ratings_matrix.T
X1 = X
X.head()

User,A00248626G99Y8GVGKKQ,A0254933A6GXCEJ2WU0K,A1006WV7OJQAWN,A101EHUSV30KDM,A101P4QD1IF90E,A102DPLUNIVSU5,A102EAFS6ZQDA3,A103S8FLFG5COQ,A103UQZW5MUI0M,A105E3ITHWEQMM,...,AZWLKUEVK9I0O,AZXP6TY9153BF,AZXPDKNG6MCDO,AZXTIJP7AUXQ1,AZYQE4YLJCLBI,AZYT0A29GC1I5,AZYVVBYSD74KP,AZZ2HXC308RW1,AZZMV4R2574OJ,AZZXCFBNEWIBQ
Item,,,,,,,,,,,,,,,,,,,,,
B000078QKW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00007GDFV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B000085HZL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00008JOQI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00008JPRZ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
#Decomposing the Matrix

SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

(7407, 10)

In [8]:
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

(7407, 7407)

In [9]:
i = X.index[random.randint(0, np.size(correlation_matrix, 0))]

product_names = list(X.index)
product_ID = product_names.index(i)
product_ID

5095

In [10]:
correlation_product_ID = correlation_matrix[product_ID]
correlation_product_ID.shape

(7407,)

In [11]:
Recommend = list(X.index[correlation_product_ID > 0.65])

# Removes the item already bought by the customer
Recommend.remove(i) 

Recommend[0:24]

['B00007GDFV',
 'B000AI9VLQ',
 'B000C47BRE',
 'B000EVEAS8',
 'B000F4KPIS',
 'B000FSIFC2',
 'B000FT9J6C',
 'B000GHMRLW',
 'B000IF0XQS',
 'B000IZVIBC',
 'B000KBMJA8',
 'B000LSJEZS',
 'B000MQ96R0',
 'B000NBIMG0',
 'B000NBIMY2',
 'B000PMKBJS',
 'B000PQMVQK',
 'B000RP9D0Q',
 'B000UJA696',
 'B000ULRGLK',
 'B000ULUS2E',
 'B000UO7GLW',
 'B000V5DJEI',
 'B000WV17QS']

In [12]:
def parse(path):
  g = gzip.open(path, 'rt', encoding='utf-8')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

review_data = getDF('Data/meta_AMAZON_FASHION.json.gz')
data_set = review_data[:30000]
data_set
    

,title,image,brand,feature,rank,date,asin,description,price,also_view,also_buy,fit,details,similar_item,tech1
0,Slime Time Fall Fest [With CDROM and Collector...,[https://images-na.ssl-images-amazon.com/image...,Group Publishing (CO),[Product Dimensions:\n \n8....,"13,052,976inClothing,Shoesamp;Jewelry(",8.70 inches,0764443682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,XCC Qi promise new spider snake preparing men'...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,"11,654,581inClothing,Shoesamp;Jewelry(",5 star,1291691480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Magical Things I Really Do Do Too!,[https://images-na.ssl-images-amazon.com/image...,Christopher Manos,[Package Dimensions:\n \n8....,"19,308,073inClothing,ShoesJewelry(",5 star,1940280001,[For the professional or amateur magician. Ro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Ashes to Ashes, Oranges to Oranges",[https://images-na.ssl-images-amazon.com/image...,Flickerlamp Publishing,[Package Dimensions:\n \n8....,"19,734,184inClothing,ShoesJewelry(",5 star,1940735033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aether & Empire #1 - 2016 First Printing Comic...,[https://images-na.ssl-images-amazon.com/image...,NaN,[Package Dimensions:\n \n10...,"10,558,646inClothing,Shoesamp;Jewelry(",5 star,1940967805,NaN,$4.50,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29997,Honeystore Women's Cap Sleeve Crystal Evening ...,[https://images-na.ssl-images-amazon.com/image...,Honeystore,[Package Dimensions:\n \n15...,"22,005,842inClothing,ShoesJewelry(",5 star,B00F5V4JJ2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29998,Honeystore Women's Cap Sleeve Crystal Evening ...,[https://images-na.ssl-images-amazon.com/image...,Honeystore,[Package Dimensions:\n \n15...,"18,933,551inClothing,ShoesJewelry(",5 star5 star (0%),B00F5V4RDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29999,AllCraft 1000x 14 Facets Resin Rhinestone Gem ...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,"5,687,062inClothing,ShoesJewelry(",jw00541,B00F5V9758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30000,Honeystore Women's Cap Sleeve Crystal Evening ...,[https://images-na.ssl-images-amazon.com/image...,Honeystore,[Package Dimensions:\n \n15...,"24,206,139inClothing,ShoesJewelry(",5 star,B00F5V4IAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
top25 = pd.DataFrame({'ID': Recommend[0:25]})

In [14]:
top25

,ID
0,B00007GDFV
1,B000AI9VLQ
2,B000C47BRE
3,B000EVEAS8
4,B000F4KPIS
5,B000FSIFC2
6,B000FT9J6C
7,B000GHMRLW
8,B000IF0XQS
9,B000IZVIBC


In [15]:
top25.merge(data_set,left_on = 'ID', right_on = 'asin')
top25

,ID
0,B00007GDFV
1,B000AI9VLQ
2,B000C47BRE
3,B000EVEAS8
4,B000F4KPIS
5,B000FSIFC2
6,B000FT9J6C
7,B000GHMRLW
8,B000IF0XQS
9,B000IZVIBC


In [16]:
Top25Recommend = top25.merge(data_set,left_on = 'ID', right_on = 'asin')

In [17]:
Top25Recommend[Top25Recommend.columns[:2]]
Top25Recommend = Top25Recommend[Top25Recommend.columns[:2]]

In [18]:
Top25Recommend.drop_duplicates().reset_index(drop=True)

,ID,title
0,B00007GDFV,Buxton Heiress Pik-Me-Up Framed Case
1,B000AI9VLQ,"Samsonite Aspire Lite Carry On Upright Suiter,..."
2,B000C47BRE,Pleaser Men's V-Creeper 509 Slip-On
3,B000EVEAS8,"Plush Microfiber Robe - Soft, Warm, and Lightw..."
4,B000F4KPIS,Costume Culture Men's Pirate Patch and Earring
5,B000FSIFC2,Boot Shapers 300 (Pack of 3 Pairs) Boot Liners
6,B000FT9J6C,14Kt Three-Color Diamond Stackable Oval-Link Ring
7,B000GHMRLW,Carhartt Men's Loose Fit Five Pocket Canvas Ca...
8,B000IF0XQS,Baby Scrubs
9,B000IZVIBC,Kids or Extra Small Size Snap On Plain Leather...
